In [ ]:
# Mounting drive to fetch Stock prediction model.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Assign model path in drive, if project is uploaded in drive.

MODEL_PATH = "/content/drive/MyDrive/AIDI_1100_02_PROJECT/LSTM_model"

# AIDI_1100_02 Group - B

## Ram Raghu Sankar 
## Shrijit Rajesh Pendse
## Varun Kadian
## Vrushabh Shah
## Oscar Alan Lozada Villa

## December 2, 2021

<p> The program's function is to scan or parse through the website "www.prnewswire.com" - to save articles for a period of time and is able to track, store and search within the articles saved trying to find stock symbols. Also, the code can retrieve data from Yahoo! Finance's API for stock symbols that appeared in the news or articles stored before. The program has the functionality to create visualisations showing the selected stock prices for the last 30 days. </p>

In [ ]:
# BeautifulSoup Documentation - https://beautiful-soup-4.readthedocs.io/en/latest/
# BeautifulSoup @ pypi.org - https://pypi.org/project/beautifulsoup4/

In [ ]:
# Please uncomment and install these packages while first time run. These packages are not available in colab by default.


# !pip install datefinder
# !pip install yfinance

In [ ]:
import re
import warnings
from collections import deque
from datetime import datetime, timedelta

import datefinder
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests
import tensorflow as tf
import yfinance as yf
from bs4 import BeautifulSoup
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [ ]:
# Initialize constants, setting the number of articles extracted from the website to 4.

ARTICLES_TO_EXTRACT_PER_DAY = 4

## Scan/Parse

<p> Scan and parse news from "newswire" website for the past two weeks </p>

In [ ]:
class Parser:
    """
    Class with methods to generate links to news releases, links to articles, parse/scrape articles and extract required information.
    """

    def __init__(self):
        pass

    def generate_links_for_date_wise_news_releases(self) -> list:
        """
        Method to generate a list of links that fetches news release of past two weeks.
        :return: list of links to get date wise news releases.
        """

        # Get start and end date to fetch articles.
        current_time = datetime.now()
        time_two_weeks_back = current_time - timedelta(days=13)
        print(f"\nParsing news from " + time_two_weeks_back.strftime("%m/%d/%Y %H:00") + " to " + current_time.strftime(
            "%m/%d/%Y %H:00") + "\n")

        # Generate a list of links that shows news release of past two weeks.
        filtered_link_list = []
        for single_date in pd.date_range(time_two_weeks_back, current_time):
            filtered_link_list.append(f"https://www.prnewswire.com/news-releases/news-releases-list/"
                                      f"?page=1&pagesize={ARTICLES_TO_EXTRACT_PER_DAY}&month={single_date.month:02}"
                                      f"&day={single_date.day:02}&year={single_date.year:04}&hour={single_date.hour:02}")

        # Display links.
        for itr in filtered_link_list:
            print(itr)

        return filtered_link_list

    def generate_links_to_articles(self, filtered_news_links_list) -> list:
        """
        Method to generate a list of links to articles.

        :param filtered_news_links_list: list of links to news releases.
        :return articles_link_list: list of article links.
        """

        print("\n\nFetching links to articles from past two weeks..\n")

        # Looping through each date wise filtered news release links and fetching links to that days articles.
        articles_link_list = []
        for i in tqdm(filtered_news_links_list):
            response = requests.get(i)
            home_page = BeautifulSoup(response.text, 'html.parser')
            # Links to articles will be in the "a" tag with class "newsreleaseconsolidatelink display-outline".
            news_release_list = list(
                home_page.find_all("a", attrs={'class': 'newsreleaseconsolidatelink display-outline'}))
            # Adding all article links to one list.
            articles_link_list.extend(
                [f"https://www.prnewswire.com{i.attrs.get('href')}" for i in news_release_list if
                 i.attrs.get("href", "")])
        print("\n\nFetching complete.\n\n")

        # Display all article links.
        for itr in articles_link_list:
            print(itr)

        print(f"\n\nNumber of articles: {len(articles_link_list)}")

        return articles_link_list

    def parse_data_from_web(self, articles_link_list) -> pd.DataFrame:
        """
        Method to loop through article links, parse article body, date and add to dataframe.
        :param articles_link_list: list of links to articles.
        :return: Parsed data in dataframe.
        """

        print("\n\nExtracting information from scrapped content..\n")

        # Define dataframe.
        data = pd.DataFrame(columns=["url", "article_date", "article_content"])

        # Looping through article links and parsing, retrieving, storing information.
        for i in tqdm(articles_link_list):
            blog = requests.get(i)
            blog_soup = BeautifulSoup(blog.text, 'html.parser')
            # Article body is present in "section tag" with class "release-body container"
            blog_body = blog_soup.find("section", attrs={'class': 'release-body container'})
            if not blog_body:
                blog_body = blog_soup.find("section", attrs={'class': 'release-body container '})
            blog_body = blog_body.text if blog_body else ""

            # Fetch article date which is present in "meta" tag.
            blog_date = blog_soup.find("meta", attrs={'name': 'date'}).attrs.get("content", "")
            matches = list(datefinder.find_dates(blog_date))
            blog_date = str(matches[0]) if matches else ""

            # Append retrieved information to dataframe.
            data = data.append({
                "url": i,
                "article_date": blog_date,
                "article_content": blog_body},
                ignore_index=True)

        print("\n\nExtraction complete. All information added to dataframe.")

        return data

## Track/Store/Search

<p> Keep track of the news by storing the parsed news (CSV, DF, DB, XLSX). For all the parsed news, search the content to find at least 2 or 3 stock symbols in a specific industry. </p>

In [ ]:
class Tracker:
    """
    Class to store extracted data and fetch stock tickers from it.
    """

    def __init__(self, data):
        self.data = data

    def store_data_as_excel(self) -> None:
        """
        Method to store data in dataframe as excel.
        :return None:
        """

        # Saving dataframe as excel with pandas to_excel function.
        self.data.to_excel(f"{datetime.now().strftime('%Y%m%d-%H%M%S')}_scrapped_data.xlsx", index=False)

    def preprocess_data(self) -> None:
        """
        Method to do preprocessing of data before extracting tickers.
        :return:
        """

        # Dropping duplicates if present.
        self.data.drop_duplicates(subset="article_content", inplace=True, ignore_index=True)

    def fetch_tickers(self) -> set:
        """
        Method to fetch tickers from all article content.
        :return: Set of tickers.
        """

        tickers = set()
        for i in range(len(self.data)):
            temp = re.findall(r':\s[A-Z]{1,5}[)]', self.data.iloc[i]["article_content"])
            for tick in temp:
                tickers.add(tick[-(len(tick) - 2):-1])
        return tickers

## Retrieve Data Web(API)

<p> Scan Yahoo! Finance page for the stock symbols selected. Retrieve the stock price and volume for the last 30 days. </p>

In [ ]:
class Retriever:
    """
    Class to retrieve stock information for tickers found in scrapped content.
    """

    def __init__(self, tickers):
        self.tickers = tickers

    def retrieve(self) -> dict:
        """
        Method to retrieve ticker prices using yahoo finance api.
        :return: Stock information as dict.
        """
        stocks = {}
        for tick in self.tickers:
            stocks[tick] = yf.Ticker(tick).history(period="YTD")
        return stocks

## Visualization

<p> Prepare colorful visualization or graphs showing the stock prices for the last 30 days. It should be a plot time-series for: </p>

1.   Volume
2.   Daily Close Price



In [ ]:
class Visualizer:
    """
    Class to display visualization of stock information.
    """

    def __init__(self, stocks):
        self.stocks = stocks

    def generate_candle_stick_visualization(self, ticker, increasing_line, decreasing_line):
        fig = go.Figure(data=[go.Candlestick(
            x=self.stocks[ticker].index, 
            open=self.stocks[ticker]['Open'],
            high=self.stocks[ticker]['High'],
            low=self.stocks[ticker]['Low'],
            close=self.stocks[ticker]['Close'], 
            increasing_line_color=increasing_line,
            decreasing_line_color=decreasing_line)
            ])
        fig.update_layout(autosize=False,
                          width=1000,
                          height=800,)
        fig.show()

    def plot_tickers(self, ticker):
        fig = plt.figure(figsize=(30, 21))
        ax1 = plt.subplot(2, 2, 1)
        plt.xticks(rotation=45)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        ax2 = plt.subplot(2, 2, 2)
        plt.xticks(rotation=45)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        ax2.yaxis.offsetText.set_fontsize(20)
        ax1.plot(self.stocks[ticker]["Close"])
        ax2.plot(self.stocks[ticker]["Volume"])

## Recommending Stock's Purchase

<p> Is the stock worth purchasing or not? </p>

In [ ]:
class StockRecommender:
    """
    Class to predict if a stock is worth purchasing.
    """

    def __init__(self, stocks):
        self.stocks = stocks

        # Loading stock recommendation model.
        self.model = tf.keras.models.load_model(f"{MODEL_PATH}")

    def model_predict(self, X):
        if np.argmax(self.model.predict(X)[-1]) == 0:
            print("Not Buy.")
        else:
            print("Buy")

    def preprocess(self, ticker):
        series = self.stocks[ticker]["Close"]
        series = series.pct_change()
        series = series.dropna()
        seq_data = []
        prev_days = deque(maxlen=100)
        for i in series:
            prev_days.append(i)
            if len(prev_days) == 100:
                seq_data.append([np.array(prev_days)])
            X = []
            for seq in seq_data:
                X.append(seq)
            X = np.array(X)
        return self.model_predict(X.reshape(-1, X.shape[2], X.shape[1]))

## Main

<p> Run all functions </p>

In [ ]:
# Web scraping.
parser_obj = Parser()

date_wise_news_releases_links = parser_obj.generate_links_for_date_wise_news_releases()
articles_links = parser_obj.generate_links_to_articles(date_wise_news_releases_links)
data = parser_obj.parse_data_from_web(articles_links)

data


Parsing news from 11/19/2021 05:00 to 12/02/2021 05:00

https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=19&year=2021&hour=05
https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=20&year=2021&hour=05
https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=21&year=2021&hour=05
https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=22&year=2021&hour=05
https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=23&year=2021&hour=05
https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=24&year=2021&hour=05
https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=25&year=2021&hour=05
https://www.prnewswire.com/news-releases/news-releases-list/?page=1&pagesize=4&month=11&day=26&year=2021&hour=05
https://www.prnewswire.com/news-release

100%|██████████| 14/14 [00:08<00:00,  1.71it/s]




Fetching complete.


https://www.prnewswire.com/news-releases/tuniu-announces-unaudited-third-quarter-2021-financial-results-301428924.html
https://www.prnewswire.com/news-releases/ssc-globeop-forward-redemption-indicator-301428849.html
https://www.prnewswire.com/news-releases/agricultural-tractor-tires-market-to-record-a-cagr-of-nearly-5-technavios-research-insights-highlight-high-labor-wages-as-key-driver-301428022.html
https://www.prnewswire.com/news-releases/hkatg-key-construction-comes-into-operation-301428949.html
https://www.prnewswire.com/news-releases/gavi-dei-gavi-la-scolca-life-style-generation-301428718.html
https://www.prnewswire.com/news-releases/an-diesem-black-friday-startet-eskute-seinen-bisher-grossten-sale-an-meistverkauften-elektrofahrradern-820945243.html
https://www.prnewswire.com/news-releases/questo-venerdi-nero-eskute-lancia-la-sua-piu-grande-vendita-di-sempre-sulle-bici-elettriche-piu-vendute-843425806.html
https://www.prnewswire.com/news-releases/ce-black-f

100%|██████████| 56/56 [00:15<00:00,  3.54it/s]



Extraction complete. All information added to dataframe.


,url,article_date,article_content
0,https://www.prnewswire.com/news-releases/tuniu...,2021-11-19 05:00:00-05:00,"\n\n\nNANJING, China, Nov. 19, 2021 /PRNewswir..."
1,https://www.prnewswire.com/news-releases/ssc-g...,2021-11-19 05:00:00-05:00,"\n\n\nWINDSOR, Conn., Nov. 19, 2021 /PRNewswir..."
2,https://www.prnewswire.com/news-releases/agric...,2021-11-19 05:00:00-05:00,"\n\n\nNEW YORK, Nov. 19, 2021 /PRNewswire/ -- ..."
3,https://www.prnewswire.com/news-releases/hkatg...,2021-11-19 05:00:00-05:00,"\n\n\nHONG KONG, Nov. 19, 2021 /PRNewswire/ --..."
4,https://www.prnewswire.com/news-releases/gavi-...,2021-11-20 05:00:00-05:00,"\n\n\nFLORENCE, Italy, Nov. 19, 2021 /PRNewswi..."
5,https://www.prnewswire.com/news-releases/an-di...,2021-11-20 04:30:00-05:00,"\n\n\nLONDON, 20. November 2021 /PRNewswire/ -..."
6,https://www.prnewswire.com/news-releases/quest...,2021-11-20 04:30:00-05:00,"\n\n\nLONDON, 20 novembre 2021 /PRNewswire/ --..."
7,https://www.prnewswire.com/news-releases/ce-bl...,2021-11-20 04:30:00-05:00,"\n\n\nLONDRES, le 20 novembre 2021 /PRNewswire..."
8,https://www.prnewswire.com/news-releases/cston...,2021-11-21 04:02:00-05:00,"\n\n\nSUZHOU, China, Nov. 21, 2021 /PRNewswire..."
9,https://www.prnewswire.com/news-releases/hisen...,2021-11-21 03:00:00-05:00,"\n\n\nQINGDAO, Chine, 21 novembre 2021 /PRNews..."


In [ ]:
# Tracking: storing data and generating tickers.

tracker_obj = Tracker(data)

tracker_obj.store_data_as_excel()
tracker_obj.preprocess_data()
tickers = tracker_obj.fetch_tickers()

tickers

{'DUB',
 'JKS',
 'LALAB',
 'NNOX',
 'OG',
 'PLTK',
 'PTNR',
 'SSNC',
 'TEDU',
 'TOUR',
 'VIOT',
 'WDH',
 'YJ',
 'ZME'}

In [ ]:
# Retrieving stock info.
retriever_obj = Retriever(tickers)

stocks = retriever_obj.retrieve()

print("\nStock Information:\n", stocks)

- DUB: No data found for this date range, symbol may be delisted
- LALAB: No data found, symbol may be delisted

Stock Information:
 {'OG':              Open   High   Low  Close   Volume  Dividends  Stock Splits
Date                                                                   
2021-05-07  11.21  11.71  6.02   7.00  2416600          0             0
2021-05-10   6.76   6.76  5.60   5.79   418300          0             0
2021-05-11   5.26   6.47  5.21   6.30   231800          0             0
2021-05-12   6.30   6.36  5.75   6.30   121600          0             0
2021-05-13   6.25   6.33  5.96   6.25   102200          0             0
...           ...    ...   ...    ...      ...        ...           ...
2021-11-24   4.86   5.03  4.70   4.86    38300          0             0
2021-11-26   4.85   4.90  4.57   4.70     4900          0             0
2021-11-29   4.63   4.63  4.20   4.40    33800          0             0
2021-11-30   4.47   4.58  4.31   4.58    23700          0           

In [ ]:
# Visualization

visualizer_obj = Visualizer(stocks)

i_cl = ["gold", "green", "cyan"]
d_cl = ["gray", "red", "black"]

widgets.interact(visualizer_obj.generate_candle_stick_visualization, ticker=stocks.keys(), increasing_line=i_cl, decreasing_line=d_cl)
widgets.interact(visualizer_obj.plot_tickers, ticker=stocks.keys())

interactive(children=(Dropdown(description='ticker', options=('OG', 'WDH', 'VIOT', 'JKS', 'TEDU', 'PLTK', 'DUB…

interactive(children=(Dropdown(description='ticker', options=('OG', 'WDH', 'VIOT', 'JKS', 'TEDU', 'PLTK', 'DUB…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

In [ ]:
# Stock recommendation
stock_recommender_obj = StockRecommender(stocks)

widgets.interact(stock_recommender_obj.preprocess, ticker=stocks.keys())

interactive(children=(Dropdown(description='ticker', options=('OG', 'WDH', 'VIOT', 'JKS', 'TEDU', 'PLTK', 'DUB…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>